# eedi_11_21_14B
* standard embedding
* resize emb to: 151668, while qwen orig is 151665. 3 new is added

In [23]:
import pandas as pd
import json
from pathlib import Path
from eedi.datasets import make_nice_df
from transformers import AutoModel, AutoTokenizer

In [10]:
root = Path("../data")
df_train = pd.read_csv(root / "train.csv")
df_train = make_nice_df(df_train)
df_mis = pd.read_csv(root / "misconception_mapping.csv")

In [ ]:
# copy prompt thing
def prompt_thing(row: pd.Series):
    template = """<instruct>Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions
<query>Question: {question}
    
SubjectName: {subject}
ConstructName: {construct}
Correct answer: {correct}
Student wrong answer: {wrong}
<response>"""
    return template.format(
        question=row["QuestionText"],
        subject=row["SubjectName"],
        construct=row["ConstructName"],
        correct=row["CorrectText"],
        wrong=row["WrongText"],
    )

# i did not implement the query max length, too lazy
df_train["prompt"] = df_train.apply(prompt_thing, axis=1)

# after this point, he concats the prompt with misconceptions?? whyy
# nah, just obfuscation

In [22]:
concat_sus = df_train["prompt"].tolist() + df_mis["MisconceptionName"].tolist()
len(concat_sus)

6957

In [25]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [28]:
model.embed_tokens

Embedding(151936, 1536)

In [30]:
# he is using special tokenier
model.resize_token_embeddings(len(tokenizer))

Embedding(151665, 1536)

In [32]:
# 
len(tokenizer) 

151665

# flag_qwen_14b
* standard embedding
* same tokenizer as 11_21_14B

In [34]:
def prompt_thing2(row: pd.Series):
    template = """Instruct: Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.
Query: ### SubjectName: {subject}
### ConstructName: {subject}
### Question: {question}
### Correct Answer: {correct}
### Misconcepte Incorrect answer: {wrong}
<response>"""
    return template.format(
        question=row["QuestionText"],
        subject=row["SubjectName"],
        construct=row["ConstructName"],
        correct=row["CorrectText"],
        wrong=row["WrongText"],
    )
df_train["prompt2"] = df_train.apply(prompt_thing2, axis=1)

In [36]:
df_train.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectChoice,CorrectText,QuestionText,WrongChoice,WrongText,MisconceptionId,QuestionId_Answer,prompt,prompt2
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\( 3 \times(2+4)-5 \),\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,D,Does not need brackets,1672,0_D,<instruct>Given a math multiple-choice problem...,Instruct: Given a math question with correct a...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",A,\( m+1 \),2142,1_A,<instruct>Given a math multiple-choice problem...,Instruct: Given a math question with correct a...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",B,\( m+2 \),143,1_B,<instruct>Given a math multiple-choice problem...,Instruct: Given a math question with correct a...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",C,\( m-1 \),2142,1_C,<instruct>Given a math multiple-choice problem...,Instruct: Given a math question with correct a...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Only\nKatie,Tom and Katie are discussing the \( 5 \) plant...,A,Only\nTom,1287,2_A,<instruct>Given a math multiple-choice problem...,Instruct: Given a math question with correct a...


# okinawa

In [37]:
def prompt_thing3(row: pd.Series):
    template = """Instruct: Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.
Query: ### SubjectName: {subject}
### ConstructName: {subject}
### Question: {question}
### Correct Answer: {correct}
### Misconcepte Incorrect answer: {wrong}"""
    return template.format(
        question=row["QuestionText"],
        subject=row["SubjectName"],
        construct=row["ConstructName"],
        correct=row["CorrectText"],
        wrong=row["WrongText"],
    )
df_train["prompt3"] = df_train.apply(prompt_thing3, axis=1)